### Imports

##### For this experiment I have modified query_processor.py, optimizer_strategy.py, experiment_utils.py, and udfs.py
##### For take TOP-K plans (K = 5)

In [1]:
import os
import time
import palimpzest as pz
from palimpzest.utils.env_helpers import load_env
from palimpzest.utils.experiment_utils import (
    load_ground_truth,
    write_results_and_plans,
)

#### Get API Keys (TogetherAI and OpenAI)

In [2]:
load_env()

## Enron (Legal Discovery) Experiment - enron-eval-50

#### Set Dataset, Groundtruth, and Output (CSVs and TXTs)

In [12]:
dataset = "../testdata/enron-eval-24"
gt_path = "../testdata/groundtruth/enron-eval.csv"
output_csv = "../experiments_results/claim-1/enron-eval-24-repro.csv"

#### Logical Plan

In [13]:
email_cols = [
    {"name": "sender", "type": str, "desc": "The email address of the sender"},
    {"name": "subject", "type": str, "desc": "The subject of the email"},
]

logical_plan = pz.TextFileDataset(id="enron", path=dataset)
logical_plan = logical_plan.sem_map(email_cols)

logical_plan = logical_plan.sem_filter(
    "The email is not quoting from a news article or an article written by someone outside of Enron",
    depends_on=["contents"],
)

logical_plan = logical_plan.sem_filter(
    'The email refers to a fraudulent scheme (i.e., "Raptor", "Deathstar", "Chewco", etc.)',
    depends_on=["contents"],
)

models = [pz.Model.LLAMA3_1_8B, pz.Model.LLAMA3_2_3B, pz.Model.LLAMA3_2_90B_V]

policy = pz.MinCostAtFixedQuality(min_quality=0.8)
config = pz.QueryProcessorConfig(
    policy=policy,
    execution_strategy="sequential",
    available_models=models
)

validator = pz.Validator(pz.Model.LLAMA3_3_70B)

#### Run Code

In [ ]:
os.makedirs(os.path.dirname(output_csv) or '.', exist_ok=True)
plan_details_dir = os.path.splitext(output_csv)[0] + "_plan_details"
os.makedirs(plan_details_dir, exist_ok=True)

start_all = time.time()
results = logical_plan.optimize_and_run(config=config, validator=validator)

total_runtime = time.time() - start_all

print(f"Received {len(results)} plan results.")

gt_data = load_ground_truth("enron", gt_path, dataset)

write_results_and_plans(
    results_iterable=results,
    logical_plan=logical_plan,
    gt_data=gt_data,
    output_csv=output_csv,
    plan_details_dir=plan_details_dir,
    workload='enron'
)

Output()

Using NAIVE PRIORS for operator sampling order



## Real Estate Listing Experiment - real-estate-eval-30

In [3]:
from palimpzest.core.lib.schemas import ImageFilepath

#### Set Dataset, Groundtruth, and Output (CSVs and TXTs)

In [4]:
dataset = "../testdata/real-estate-eval-15"
gt_path = "../testdata/groundtruth/real-estate-eval-15.csv"
output_csv = "../experiments_results/claim-1/real-estate-eval-15-repro.csv"


# dataset = "../testdata/real-estate-eval-tiny"
# gt_path = "../testdata/groundtruth/real-estate-eval-tiny.csv"
# output_csv = "../experiments_results/claim-1/real-estate-eval-tiny-repro.csv"

#### Logical Plan

In [5]:
real_estate_listing_cols = [
    {"name": "listing", "type": str, "desc": "The name of the listing"},
    {"name": "text_content", "type": str, "desc": "The content of the listing's text description"},
    {"name": "image_filepaths", "type": list[ImageFilepath], "desc": "A list of the filepaths for each image of the listing"},
]

real_estate_text_cols = [
    {"name": "address", "type": str, "desc": "The address of the property"},
    {"name": "price", "type": int | float, "desc": "The listed price of the property"},
]

real_estate_image_cols = [
    {"name": "is_modern_and_attractive", "type": bool, "desc": "True if the home interior design is modern and attractive and False otherwise"},
    {"name": "has_natural_sunlight", "type": bool, "desc": "True if the home interior has lots of natural sunlight and False otherwise"},
]

FAR_AWAY_ADDRS = [
    "Melcher St",
    "Sleeper St",
    "437 D St",
    "Seaport Blvd",
    "50 Liberty Dr",
    "Telegraph St",
    "Columbia Rd",
    "E 6th St",
    "E 7th St",
    "E 5th St",
]

In [6]:
class RealEstateListingDataset(pz.IterDataset):
    def __init__(self, listings_dir):
        super().__init__(id="real-estate", schema=real_estate_listing_cols)
        self.listings_dir = listings_dir
        self.listings = sorted(os.listdir(self.listings_dir))
        self.listings = [file for file in self.listings if not file.startswith(".")]

    def __len__(self):
        return len(self.listings)

    def __getitem__(self, idx: int):
        # get listing
        listing = self.listings[idx]

        # get fields
        image_filepaths, text_content = [], None
        listing_dir = os.path.join(self.listings_dir, listing)
        for file in os.listdir(listing_dir):
            if file.endswith(".txt"):
                with open(os.path.join(listing_dir, file), "rb") as f:
                    text_content = f.read().decode("utf-8")
            elif file.endswith(".png"):
                image_filepaths.append(os.path.join(listing_dir, file))

        # construct and return dictionary with fields
        return {"listing": listing, "text_content": text_content, "image_filepaths": image_filepaths}

def within_two_miles_of_mit(record: dict):
    # NOTE: I'm using this hard-coded function so that folks w/out a
    #       Geocoding API key from google can still run this example
    try:
        return not any([street.lower() in record["address"].lower() for street in FAR_AWAY_ADDRS])
    except Exception:
        return False


def in_price_range(record: dict):
    try:
        price = record["price"]
        if isinstance(price, str):
            price = price.strip()
            price = int(price.replace("$", "").replace(",", ""))
        return 6e5 < price <= 2e6
    except Exception:
        return False

In [ ]:
logical_plan = RealEstateListingDataset(dataset)

logical_plan = logical_plan.sem_map(real_estate_text_cols, depends_on="text_content")

logical_plan = logical_plan.sem_map(real_estate_image_cols, depends_on="image_filepaths")

logical_plan = logical_plan.sem_filter(
    "The interior is modern and attractive, and has lots of natural sunlight",
    depends_on=["is_modern_and_attractive", "has_natural_sunlight"],
)

logical_plan = logical_plan.filter(within_two_miles_of_mit, depends_on="address")

logical_plan = logical_plan.filter(in_price_range, depends_on="price")

models = [pz.Model.LLAMA3_2_90B_V, pz.Model.GPT_4o]

policy = pz.MinCostAtFixedQuality(min_quality=0.8)
config = pz.QueryProcessorConfig(
    policy=policy,
    execution_strategy="sequential",
    available_models=models
)

validator = pz.Validator(pz.Model.GPT_4o)

#### Run Code

In [ ]:
os.makedirs(os.path.dirname(output_csv) or '.', exist_ok=True)
plan_details_dir = os.path.splitext(output_csv)[0] + "_plan_details"
os.makedirs(plan_details_dir, exist_ok=True)

start_all = time.time()
results = logical_plan.optimize_and_run(config=config, validator=validator)
# results = logical_plan.run(config=config)

total_runtime = time.time() - start_all

print(f"Received {len(results)} plan results.")

gt_data = load_ground_truth("real-estate", gt_path, dataset)

write_results_and_plans(
    results_iterable=results,
    logical_plan=logical_plan,
    gt_data=gt_data,
    output_csv=output_csv,
    plan_details_dir=plan_details_dir,
    workload='real-estate'
)

## Medical Schema Eval Experiment - biofabric-matchingfrom palimpzest.utils.udfs import xls_to_tables

In [3]:
from palimpzest.utils.udfs import xls_to_tables
!pip install openpyxl

#### Set Dataset, Groundtruth, and Output (CSVs and TXTs)

In [4]:
dataset = "../testdata/biofabric-matching"
gt_path = "../testdata/target_matching.csv"
output_csv = "../experiments_results/claim-1/target_matching.csv"

#### Logical Plan

In [5]:
case_cols = [
    {"name": "case_submitter_id", "type": str, "desc": "The ID of the case"},
    {"name": "age_at_diagnosis", "type": int | float, "desc": "The age of the patient at the time of diagnosis"},
    {"name": "race", "type": str, "desc": "An arbitrary classification of a taxonomic group that is a division of a species."},
    {"name": "ethnicity", "type": str, "desc": "Whether an individual describes themselves as Hispanic or Latino or not."},
    {"name": "gender", "type": str, "desc": "Text designations that identify gender."},
    {"name": "vital_status", "type": str, "desc": "The vital status of the patient"},
    {"name": "ajcc_pathologic_t", "type": str, "desc": "Code of pathological T (primary tumor) to define the size or contiguous extension of the primary tumor (T), using staging criteria from the American Joint Committee on Cancer (AJCC)."},
    {"name": "ajcc_pathologic_n", "type": str, "desc": "The codes that represent the stage of cancer based on the nodes present (N stage) according to criteria based on multiple editions of the AJCC's Cancer Staging Manual."},
    {"name": "ajcc_pathologic_stage", "type": str, "desc": "The extent of a cancer, especially whether the disease has spread from the original site to other parts of the body based on AJCC staging criteria."},
    {"name": "tumor_grade", "type": int | float, "desc": "Numeric value to express the degree of abnormality of cancer cells, a measure of differentiation and aggressiveness."},
    {"name": "tumor_focality", "type": str, "desc": "The text term used to describe whether the patient's disease originated in a single location or multiple locations."},
    {"name": "tumor_largest_dimension_diameter", "type": int | float, "desc": "The tumor largest dimension diameter."},
    {"name": "primary_diagnosis", "type": str, "desc": "Text term used to describe the patient's histologic diagnosis, as described by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O)."},
    {"name": "morphology", "type": str, "desc": "The Morphological code of the tumor, as described by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O)."},
    {"name": "tissue_or_organ_of_origin", "type": str, "desc": "The text term used to describe the anatomic site of origin, of the patient's malignant disease, as described by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O)."},
    {"name": "study", "type": str, "desc": "The last name of the author of the study, from the table name"},
    {"name": "filename", "type": str, "desc": "The name of the file the record was extracted from"}
]

table_cols = [
    {"name": "rows", "type": list[str], "desc": "The rows of the table"},
    {"name": "header", "type": list[str], "desc": "The header of the table"},
    {"name": "name", "type": str, "desc": "The name of the table"},
    {"name": "filename", "type": str, "desc": "The name of the file the table was extracted from"}
]

In [6]:
plan = pz.XLSFileDataset(id="medical", path=dataset)

logical_plan = plan.add_columns(xls_to_tables, cols=table_cols, cardinality=pz.Cardinality.ONE_TO_MANY)

logical_plan = logical_plan.sem_filter("The rows of the table contain the patient age")

logical_plan = logical_plan.sem_add_columns(case_cols, cardinality=pz.Cardinality.ONE_TO_MANY)

models = [pz.Model.LLAMA3_1_8B, pz.Model.LLAMA3_2_3B, pz.Model.LLAMA3_2_90B_V]

policy = pz.MinCostAtFixedQuality(min_quality=0.8)
config = pz.QueryProcessorConfig(
    policy=policy,
    execution_strategy="sequential",
    available_models=models
)

validator = pz.Validator(pz.Model.LLAMA3_3_70B)

/tmp/ipykernel_1347/3773151331.py:3: DeprecationWarning: we are renaming this function to `map` and deprecating `add_columns` in the next release of PZ. To update your code, simply change your calls from `.add_columns(...)` to `.map(...)`. The function arguments will stay the same.
  logical_plan = plan.add_columns(xls_to_tables, cols=table_cols, cardinality=pz.Cardinality.ONE_TO_MANY)
/tmp/ipykernel_1347/3773151331.py:7: DeprecationWarning: we are renaming this function to `sem_map` and deprecating `sem_add_columns` in the next release of PZ. To update your code, simply change your calls from `.sem_add_columns(...)` to `.sem_map(...)`. The function arguments will stay the same.
  logical_plan = logical_plan.sem_add_columns(case_cols, cardinality=pz.Cardinality.ONE_TO_MANY)


In [7]:
os.makedirs(os.path.dirname(output_csv) or '.', exist_ok=True)
plan_details_dir = os.path.splitext(output_csv)[0] + "_plan_details"
os.makedirs(plan_details_dir, exist_ok=True)

start_all = time.time()
results = logical_plan.optimize_and_run(config=config, validator=validator)
# results = logical_plan.run(config = config)

total_runtime = time.time() - start_all

print(f"Received {len(results)} plan results.")

gt_data = load_ground_truth("medical-schema-matching", gt_path, dataset)

write_results_and_plans(
    results_iterable=results,
    logical_plan=logical_plan,
    gt_data=gt_data,
    output_csv=output_csv,
    plan_details_dir=plan_details_dir,
    workload='medical-schema-matching'
)

Output()

Using NAIVE PRIORS for operator sampling order



Total opt. time: 807.56s
Total opt. cost: $0.0630


Output()

Output()

...

Output()

Total time: 210.81s
Total cost: $0.0062



Output()

Total time: 159.50s
Total cost: $0.0080



Total time: 255.10s
Total cost: $0.0071
Received 5 plan results.
Saved results to ../experiments_results/claim-1/target_matching.csv
Plan details saved in ../experiments_results/claim-1/target_matching_plan_details
